In [335]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [336]:
base = pd.read_csv('/content/census.csv')

In [337]:
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [338]:
previsores = base.iloc[:,0:14].values

In [339]:
previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [340]:
saida = base.iloc[:,14].values

In [341]:
saida

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

Converter as variaveis categoricas em numericas


In [342]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()

In [343]:
previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [344]:
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [345]:
previsores

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.1740e+03, 0.0000e+00,
        4.0000e+01],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.3000e+01],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+01],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+01],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        2.0000e+01],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.5024e+04, 0.0000e+00,
        4.0000e+01]])

In [346]:
labelencorder_saida = LabelEncoder()
saida = labelencorder_saida.fit_transform(saida)

In [347]:
saida

array([0, 0, 0, ..., 0, 0, 1])

In [348]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
previsores = scaler.fit_transform(previsores)

In [349]:
previsores

array([[0.        , 0.        , 0.        , ..., 0.29437338, 0.        ,
        3.23959063],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.05286695],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        3.23959063],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        3.23959063],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.61979531],
       [0.        , 0.        , 0.        , ..., 2.03434482, 0.        ,
        3.23959063]])

In [350]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(previsores, saida, test_size = 0.2, random_state = 5)

In [351]:
X_train.shape

(26048, 108)

In [352]:
y_train.shape

(26048,)

In [354]:
classifier = tf.keras.models.Sequential()
classifier.add(tf.keras.layers.Dense(units=100, activation='relu', input_shape=(108, )))
classifier.add(tf.keras.layers.Dense(units=100, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [353]:
classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               10900     
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 21,101
Trainable params: 21,101
Non-trainable params: 0
_________________________________________________________________


In [355]:
classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [356]:
epochs_hist = classifier.fit(X_train, y_train, epochs=200)

Epoch 1/200
814/814 [==============================] - 2s 2ms/step - loss: 0.4115 - accuracy: 0.8026
Epoch 2/200
814/814 [==============================] - 1s 1ms/step - loss: 0.3276 - accuracy: 0.8436
Epoch 3/200
814/814 [==============================] - 1s 2ms/step - loss: 0.3147 - accuracy: 0.8539
Epoch 4/200
814/814 [==============================] - 1s 2ms/step - loss: 0.3083 - accuracy: 0.8574
Epoch 5/200
814/814 [==============================] - 1s 1ms/step - loss: 0.3044 - accuracy: 0.8571
Epoch 6/200
814/814 [==============================] - 1s 2ms/step - loss: 0.2930 - accuracy: 0.8601
Epoch 7/200
814/814 [==============================] - 1s 2ms/step - loss: 0.2889 - accuracy: 0.8637
Epoch 8/200
814/814 [==============================] - 1s 1ms/step - loss: 0.2842 - accuracy: 0.8665
Epoch 9/200
814/814 [==============================] - 1s 1ms/step - loss: 0.2766 - accuracy: 0.8691
Epoch 10/200
814/814 [==============================] - 1s 1ms/step - loss: 0.2731 - accura

In [357]:
entrada = [[39,' State-gov',77516,' Bachelors',13,' Never-married',' Adm-clerical',' Not-in-family',' White',' Male',2174,0,40,' United-States']]

In [358]:
entradaencoded = onehotencorder.transform(entrada)

In [359]:
entradaescalonada = scaler.transform(entradaencoded)

In [360]:
previsao = classifier.predict(entradaescalonada)

In [361]:
labelencorder_saida.inverse_transform(previsao[0].astype(int))

array([' <=50K'], dtype=object)

In [362]:
base.loc[base['income'] == ' >50K']

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
10,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K
11,30,State-gov,141297,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32539,71,?,287372,Doctorate,16,Married-civ-spouse,?,Husband,White,Male,0,0,10,United-States,>50K
32545,39,Local-gov,111499,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,20,United-States,>50K
32554,53,Private,321865,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K


In [363]:
entrada = base.loc[base['income'] == ' >50K']

In [364]:
entrada = base.loc[base['income'] == ' >50K'].iloc[1:2,0:14]

In [365]:
entrada

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States


In [366]:
entradaencoded = onehotencorder.transform(entrada)

In [368]:
entradaescalonada = scaler.transform(entradaencoded)

In [369]:
previsao = classifier.predict(entradaescalonada)
previsao

array([[1.]], dtype=float32)

In [371]:
labelencorder_saida.inverse_transform(previsao[0].astype(int))[0]

' >50K'

Salvar Modelo e Serializar os Encoders


In [376]:
classifier.save('modelosalvo.h5')

In [377]:
novo_modelo = tf.keras.models.load_model('modelosalvo.h5')

In [378]:
novo_modelo.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               10900     
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 21,101
Trainable params: 21,101
Non-trainable params: 0
_________________________________________________________________


In [379]:
loss, acc = novo_modelo.evaluate(previsores, saida, verbose=2)
print('Modelo Restaurado, Precisão: {:5.2f}%'.format(100 * acc))

1018/1018 - 1s - loss: 0.4641 - accuracy: 0.9277
Modelo Restaurado, Precisão: 92.77%


In [383]:
from joblib import dump, load
dump(onehotencorder, 'dicionario.joblib')

['dicionario.joblib']

In [384]:
onehotencodernovo = load('dicionario.joblib')

In [385]:
onehotencodernovo

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('OneHot',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=True),
                                 [1, 3, 5, 6, 7, 8, 9, 13])],
                  verbose=False)

In [386]:
dump(labelencorder_saida, 'saidadicionario.joblib')

['saidadicionario.joblib']

In [387]:
labelencorder_saidanovo = load('saidadicionario.joblib')

In [388]:
labelencorder_saidanovo

LabelEncoder()